## Assignment 3_ Part 4_Question 1
- Use 'movies_awards' data set.
- You are supposed to extract data from the awards column in this dataset and split it into several columns. An example is given below.
- The awards has details of wins, nominations in general and also wins and nominations in certain categories(e.g. Oscar, BAFTA etc.)
- You are supposed to create a win and nominated column (these 2 columns contain total number of wins and nominations) and other columns that extract the number of wins and nominations for each category of award. 
- If a movie has 2 Oscar nominations and 4 Oscar won, the columns Oscar_Awards_Won should have value 4 and Oscar_Awards_Nominated should have value 2. You should also have a total won and nominated column which aggregates all the awards (won or nominated). 
- Create two separate columns for each award category (won and nominated).
- Write your output to a csv file. (Sample output is given in next page).

In [1]:
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer as wnl
import re

In [2]:
df = pd.read_csv('Data/movies_awards.csv')

In [3]:
df=df[df.Awards.notnull()] 
df=df.reset_index(drop=True)

In [4]:
df.head()

,totalSeasons,Plot,Rated,Response,Language,Title,Country,Writer,Metascore,imdbRating,...,Released,Actors,Year,Genre,Awards,Runtime,Type,Poster,imdbVotes,imdbID
0,none,A historical classic drama in three acts; reto...,NaN,True,Portuguese,Who Are You?,Portugal,João Botelho; Almeida Garrett (play),NaN,6.1,...,23 Nov 2001,Patrícia Guerreiro; Suzana Borges; Rui Morisso...,2001,Drama; History,1 win & 2 nominations.,112 min,movie,http://ia.media-imdb.com/images/M/MV5BMTM1Mjdi...,189.0,tt0210930
1,none,A father (Williams) suspects his son (Jackson)...,R,True,English,Skeletons in the Closet,USA,Donna Powers; Wayne Powers,NaN,5.4,...,24 Jul 2001,Treat Williams; Linda Hamilton; Jonathan Jacks...,2001,Crime; Drama; Mystery,1 win.,100 min,movie,http://ia.media-imdb.com/images/M/MV5BMjA0NjA2...,603.0,tt0212513
2,none,Four perky; clueless cheerleaders from the Hap...,R,True,English,Cheerleader Ninjas,USA,Kevin Campbell,NaN,2.3,...,25 Nov 2003,Kira Reed Lorsch; Angela Brubaker; Jeff Nichol...,2002,Action; Comedy,1 nomination.,91 min,movie,http://ia.media-imdb.com/images/M/MV5BMjA3MTQ1...,881.0,tt0235280
3,none,Sandrine; a woman in her thirties gets tired o...,NaN,True,French,The Girl from Paris,France,Christian Carion; Eric Assous,64.0,6.8,...,05 Sep 2001,Michel Serrault; Mathilde Seigner; Jean-Paul R...,2001,Comedy; Drama,3 wins & 2 nominations.,103 min,movie,https://images-na.ssl-images-amazon.com/images...,1715.0,tt0240149
4,none,A rivalry develops between a showbiz veteran a...,R,True,English,The Final Curtain,UK; USA,John Hodge,NaN,5.8,...,12 Oct 2002,Peter O'Toole; Aidan Gillen; Adrian Lester; Ju...,2002,Drama,1 win & 1 nomination.,85 min,movie,http://ia.media-imdb.com/images/M/MV5BOGI4MmIw...,466.0,tt0243904


In [5]:
temp_df = df.loc[:,['Title','Awards']]
temp_df.head(10)

,Title,Awards
0,Who Are You?,1 win & 2 nominations.
1,Skeletons in the Closet,1 win.
2,Cheerleader Ninjas,1 nomination.
3,The Girl from Paris,3 wins & 2 nominations.
4,The Final Curtain,1 win & 1 nomination.
5,Crazy/Beautiful,5 nominations.
6,The Anniversary Party,2 wins & 5 nominations.
7,The Quiet American,Nominated for 1 Oscar. Another 13 wins & 13 no...
8,Like Cat and Mouse,5 wins & 4 nominations.
9,Punch-Drunk Love,Nominated for 1 Golden Globe. Another 14 wins ...


In [6]:
check_list_1 = temp_df['Awards'].apply(lambda x:re.findall(r"for \d+ ([A-Z a-z]+).",x)[0] if len(re.findall(r"for \d+ (\w.)",x))!=0 else "").unique()

In [7]:
check_list_2 = temp_df['Awards'].apply(lambda x:re.findall(r"Won \d+ ([A-Z a-z]+).",x)[0] if len(re.findall(r"Won \d+ (\w.)",x))!=0 else "").unique()

In [8]:
conbine_list = np.append(check_list_1, check_list_2)
check_list = []
'''wnl = WordNetLemmatizer()
for award in conbine_list:
    if award != "" and award not in check_list:
        check_list.append(wnl.lemmatize(award))'''     # Doesn't work for Proper nouns !

for award in conbine_list:
    if award != "": 
        if award.endswith('s'):
            char_list = list(award)       
            char_list.pop()             
            word = ''.join(char_list)
            if word not in check_list:
                check_list.append(word)
        else:
            if award not in check_list:
                check_list.append(award)      

In [9]:
check_list

['Oscar', 'Golden Globe', 'Primetime Emmy', 'BAFTA Film Award']

### Split logic part

In [10]:
# finding digits which preceded 'win' and 'Won'
temp_df['Total_Awards_Won']=temp_df.Awards.apply(lambda x:(int(re.findall(r"(\d+) win",x)[0]) if len(re.findall(r"(\d+) win",x))!=0 else 0) 
                                                 + (int(re.findall(r"Won (\d+)",x)[0]) if len(re.findall(r"Won (\d+)",x))!=0 else 0))

In [11]:
#Creating Column Awards_nominated and adding values to it
temp_df['Total_Awards_Nominated']=temp_df.Awards.apply(lambda x:(int(re.findall(r"(\d+) nom",x)[0]) if len(re.findall(r"(\d+) nom",x))!=0 else 0)
                                        + (int(re.findall(r"Nominated for (\d+) Pri",x)[0]) if len(re.findall(r"Nominated for (\d+) Pri",x))!=0 else 0)
                                        + (int(re.findall(r"Nominated for (\d+) Osc",x)[0]) if len(re.findall(r"Nominated for (\d+) Osc",x))!=0 else 0)
                                        + (int(re.findall(r"Nominated for (\d+) Gol",x)[0]) if len(re.findall(r"Nominated for (\d+) Gol",x))!=0 else 0)
                                        + (int(re.findall(r"Nominated for (\d+) BAFTA",x)[0]) if len(re.findall(r"Nominated for (\d+) BAFTA",x))!=0 else 0))

In [12]:
#Creating Columns and adding values to them
temp_df['Prime_Awards_Nominated']=temp_df.Awards.apply(lambda x:(int(re.findall(r"Nominated for (\d+) Pri",x)[0]) if len(re.findall(r"Nominated for (\d+) Pri",x))!=0 else 0))
temp_df['Oscar_Awards_Nominated']=temp_df.Awards.apply(lambda x:(int(re.findall(r"Nominated for (\d+) Osc",x)[0]) if len(re.findall(r"Nominated for (\d+) Osc",x))!=0 else 0))
temp_df['Golden_Globe_Awards_Nominated']=temp_df.Awards.apply(lambda x:(int(re.findall(r"Nominated for (\d+) Gol",x)[0]) if len(re.findall(r"Nominated for (\d+) Gol",x))!=0 else 0))
temp_df['BAFTA_Awards_Nominated']=temp_df.Awards.apply(lambda x:(int(re.findall(r"Nominated for (\d+) BAFTA",x)[0]) if len(re.findall(r"Nominated for (\d+) BAFTA",x))!=0 else 0))


In [13]:
#Creating Columns and adding values to them
temp_df['Prime_Awards_Won']=temp_df.Awards.apply(lambda x:(int(re.findall(r"Won (\d+) Pri",x)[0]) if len(re.findall(r"Won (\d+) Pri",x))!=0 else 0))
temp_df['Oscar_Awards_Won']=temp_df.Awards.apply(lambda x:(int(re.findall(r"Won (\d+) Osc",x)[0]) if len(re.findall(r"Won (\d+) Osc",x))!=0 else 0))
temp_df['Golden_Globe_Awards_Won']=temp_df.Awards.apply(lambda x:(int(re.findall(r"Won (\d+) Gol",x)[0]) if len(re.findall(r"Won (\d+) Gol",x))!=0 else 0))
temp_df['BAFTA_Awards_Won']=temp_df.Awards.apply(lambda x:(int(re.findall(r"Won (\d+) BAFTA",x)[0]) if len(re.findall(r"Won (\d+) BAFTA",x))!=0 else 0))

In [14]:
temp_df.head()

,Title,Awards,Total_Awards_Won,Total_Awards_Nominated,Prime_Awards_Nominated,Oscar_Awards_Nominated,Golden_Globe_Awards_Nominated,BAFTA_Awards_Nominated,Prime_Awards_Won,Oscar_Awards_Won,Golden_Globe_Awards_Won,BAFTA_Awards_Won
0,Who Are You?,1 win & 2 nominations.,1,2,0,0,0,0,0,0,0,0
1,Skeletons in the Closet,1 win.,1,0,0,0,0,0,0,0,0,0
2,Cheerleader Ninjas,1 nomination.,0,1,0,0,0,0,0,0,0,0
3,The Girl from Paris,3 wins & 2 nominations.,3,2,0,0,0,0,0,0,0,0
4,The Final Curtain,1 win & 1 nomination.,1,1,0,0,0,0,0,0,0,0


In [15]:
temp_df.to_csv('outputs/awards_split.csv', index=False)

## Code referenced from Raju Singh.